In [ ]:
from pandas import get_dummies
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

## Get Data


In [ ]:
import duckdb
con = duckdb.connect('my-db.duckdb')
df = con.execute("SELECT * FROM penguins").fetchdf().dropna()
con.close()

df.head(3)

## Define Model and Fit


In [ ]:
X = get_dummies(df[['bill_length_mm', 'species', 'sex']], drop_first = True)
y = df['body_mass_g']

model = LinearRegression().fit(X, y)

## Get some information


In [ ]:
print(f"R^2 {model.score(X,y)}")
print(f"Intercept {model.intercept_}")
print(f"Columns {X.columns}")
print(f"Coefficients {model.coef_}")

## Turn into Vetiver Model


In [ ]:
from vetiver import VetiverModel
v = VetiverModel(model, model_name='penguin_model', prototype_data=X)

## Save to Board


In [ ]:
from pins import board_folder
from vetiver import vetiver_pin_write

model_board = board_folder("data/model", allow_pickle_read = True)
vetiver_pin_write(model_board, v)

## Turn model into API


In [ ]:
from vetiver import VetiverAPI
app = VetiverAPI(v, check_prototype = True)

if __name__ == "__main__":
  app.run(port=8080)

In [ ]:
from vetiver import prepare_docker
prepare_docker(model_board, 'penguin_model')